## Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

## Unsloth 파인튜닝


In [ ]:
# !pip install datasets -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import os
from datasets import load_dataset

import torch

from unsloth import FastModel
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_data_formats

from transformers import TextStreamer


# 사용가능한 모델들
fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-07 07:14:09 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
# 모델, 토크나이저 불러오기
model, tokenizer = FastModel.from_pretrained(
    model_name = fourbit_models[1],
    max_seq_length = 2048,   # 모델이 처리할 수 있는 시퀀스 길이
    load_in_4bit = True,     # 4비트 양자화로 메모리 줄이기
    load_in_8bit = False,    # 8비트 양자화로 메모리는 4비트의 2배
    full_finetuning = False, # 풀파인튜닝 여부 LoRA를 사용할 거면 False
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

# 용어 설명
- **PEFT(Parameter-Efficient-Fine-Tuning)** : 효과적인 파인튜닝을 위한 기술 LoRA는 PEFT의 기술 중 하나
- **LoRA (Low-Rank Adaptation)** : 모델 전체를 파인튜닝하지 않고, 작은 어댑터를 추가하여 모델을 효율적으로 파인튜닝하는 기술(메모리 사용량을 줄이고, 속도를 높힐 수 있음)   





In [ ]:
#  get_peft_model : Unsloth의 LoRA 설정 함수
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # 시각 기반인가? False
    finetune_language_layers   = True,  # 언어 기반인가? True
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # MLP (Multi-Layer Perceptron) 파인튜닝에 중요한 역할 항상 True 유지

    r = 8,            # LoRA 어댑터의 랭크(차원), 높을수록 모델의 설명력이 좋아지지만 과적합 됨(일반적으로 8이나 16)
    lora_alpha = 8,   # LoRA 어댑터의 스케일링(업데이트 크기), 보통 r과 같은값(최소가 r값)
    lora_dropout = 0, # LoRA 어댑터의 드롭아웃 비율(과적합 방지) 0이나 0.1
    bias = "none",    # LoRA어댑터의 편향 추가여부 "none", "lora_only"(lora에만 추가), "all" 모두 추가
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [ ]:
# @title 기본 모델 답변 테스트
question = "갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘"
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : question,
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # 대화형 모델에게 생성을 알리는 신호
)
outputs = model.generate(
    # 토크나이저가 텍스트를 숫자로 변환, return_tensors='pt'는 PyTorch텐서로 변환
    **tokenizer([text], return_tensors = "pt").to("cuda"), # to.("cuda")는 PyTorch텐서를 CUDA(GPU)장치로 이동
    max_new_tokens = 500, # 출력 길이
    # Gemma-3의 권장 설정
    temperature = 1.0, top_p = 0.95, top_k = 64,
)

# 출력 결과 확인
response = tokenizer.batch_decode(outputs)

In [ ]:
# 파인튜닝 전 gemma3모델
user_input = re.search(r'<start_of_turn>user\n(.*?)<end_of_turn>', response[0]).group(1).strip()
print(f"""질문 : \n {user_input}""")

model_output = re.search(r'<start_of_turn>model\n(.*?)\n\n', response[0]).group(1).strip()
print(f"""답변 : \n {model_output}""")

질문 : 
 갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘
답변 : 
 갈현동에서 보증금 5천만 원, 월세 200만 원 이하, 50m² 이상 규모의 카페 창업 공간을 찾는 데 도움을 드리겠습니다. 현재 시세와 시장 상황을 고려하여 현실적인 옵션을 찾아보겠습니다.


In [ ]:
# 데이터 형식을 표준 형식으로 변환
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/data/ft_prompt.jsonl", split='train')
dataset = standardize_data_formats(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['messages'],
    num_rows: 108
})

In [ ]:
dataset['messages'][0]

[{'role': 'system',
  'content': "\nYou are a commercial district analysis expert specializing in Seoul, South Korea. \nYour primary task is to provide information on the monthly rent, deposit, area, and floor of properties that match the user's desired criteria. \nPlease provide responses to user inquiries in Korean.\n[user input] is a user question and [assident output] is an answer format. Please answer according to the answer format.\n\n\n[user input]\n서울에서 보증금 3천 만원, 월세 80 만원 이하로 음식점을 위한 매물을 구할 수 있을까?\n고층이어도 상관없고, 10m²보다는 컸으면 좋겠어.\n\n[assistant output]\n📍 음식점 창업 매물 추천\n\n원하는 조건에 맞는 매물을 찾는다면, 중구는 어떤가요?\n\n───────────────────────────\n\n🏠 매물 1\n위치: 중구 필동2가\n보증금: 1000 만원\n월세: 80 만원\n권리금: 1500 만원\n면적: 23.1m²\n층수: 1층\n설명: 상권이잘형성되어있어서유동인구가많아유입되는손님이꾸준히있어서매출이꾸준히나오는매장.\n추천 이유: 원하는 조건에 맞는 매물입니다.\n\n🔗 매물 보러가기: [클릭](https://www.zigbang.com/home/store/items/571420)\n\n───────────────────────────\n\n🏠 매물 2\n위치: 중구 쌍림동\n보증금: 0 만원\n월세: 75 만원\n권리금: 6000 만원\n면적: 56.2 m²\n층수: 1층\n설명: 역초역세권카페매물\n추천 이

In [ ]:
# 데이터프레임에 text열을 추가해서 gemma3에 맞는 데이터 형식 추가
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["messages"]) # 토크나이저 형식대로 바꿔라
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True) # 함수 삽입

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
# 파인튜닝용 데이터
"""
<bos> : 대화의 시작을 나타냅니다.
<start_of_turn> : 사용자 또는 모델의 발화 시작을 나타냅니다.
user: 사용자의 발화를 나타냅니다.
<end_of_turn> : 사용자 또는 모델의 발화 종료를 나타냅니다.
model: 모델의 발화를 나타냅니다.
"""

dataset[0]["text"]

"<bos><start_of_turn>user\n\nYou are a commercial district analysis expert specializing in Seoul, South Korea. \nYour primary task is to provide information on the monthly rent, deposit, area, and floor of properties that match the user's desired criteria. \nPlease provide responses to user inquiries in Korean.\n[user input] is a user question and [assident output] is an answer format. Please answer according to the answer format.\n\n\n[user input]\n서울에서 보증금 3천 만원, 월세 80 만원 이하로 음식점을 위한 매물을 구할 수 있을까?\n고층이어도 상관없고, 10m²보다는 컸으면 좋겠어.\n\n[assistant output]\n📍 음식점 창업 매물 추천\n\n원하는 조건에 맞는 매물을 찾는다면, 중구는 어떤가요?\n\n───────────────────────────\n\n🏠 매물 1\n위치: 중구 필동2가\n보증금: 1000 만원\n월세: 80 만원\n권리금: 1500 만원\n면적: 23.1m²\n층수: 1층\n설명: 상권이잘형성되어있어서유동인구가많아유입되는손님이꾸준히있어서매출이꾸준히나오는매장.\n추천 이유: 원하는 조건에 맞는 매물입니다.\n\n🔗 매물 보러가기: [클릭](https://www.zigbang.com/home/store/items/571420)\n\n───────────────────────────\n\n🏠 매물 2\n위치: 중구 쌍림동\n보증금: 0 만원\n월세: 75 만원\n권리금: 6000 만원\n면적: 56.2 m²\n층수: 1층\n설명: 역초역세권카페매물\n추천 이유: 월세는 

In [ ]:
# 파인튜닝 모델 정의
from trl import SFTTrainer, SFTConfig
ft_trainer1 = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",     # 데이터셋에서 대화가 담긴 열 이름
        per_device_train_batch_size = 2, # 장치(CPU, GPU, TPU)가 한 번에 학습하는 데이터 개수
        gradient_accumulation_steps = 4, # 그라디언트 4번을 가중 평균하여 한 번만 가중치 업데이트(2배치x4스탭 = 8배치와 같은 효과), gpu메모리 절약
        warmup_steps = 5,                # 훈련 초기에 학습률을 점진적으로 늘려 안정적인 학습 유도(0~5회 학습에서 점차 learning_rate 증가)
        num_train_epochs = 3,            # 전체 데이터를 모두 학습하는 횟수 (max_steps와 같이 넣으면, 빨리 끝나는 조건을 따름)
        # max_steps = 30,                # batch_size x maxsteps 만큼의 데이터양 학습 진행 (전체 데이터를 다 안 봐도 학습 횟수만큼만 학습하고 종료)
        learning_rate = 2e-4,            # 2e-4도 큰 폭임 긴 훈련은 2e-5처럼 더 작은 값을 사용
        logging_steps = 1,               # 훈련 상황 기록 빈도, 1이면 매 단계에 로그 기록
        optim = "adamw_8bit",            # 옵티마이저 - 아담w 8비트(메모리사용량을 줄이고 훈련속도를 높임)
        weight_decay = 0.01,             # 가중치를 감소시켜 과적합 방지
        lr_scheduler_type = "linear",    # 학습률 감소 방식(후반에 학습률을 줄여 최적값에 안정적으로 수렴)
        seed = 3407,                     # 랜덤스테이트같은거
        report_to = "none",              # 훈련 결과 확인 여부 "WandB"로 하면 Weights와 Baises 확인 가능
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",      # 체크포인트 저장 디렉토리
        save_strategy="best",           # 단계별로 저장
        metric_for_best_model="loss",
        save_steps=100,                  # 100단계마다 저장
        save_total_limit=3,              # 최대 3개의 체크포인트 저장
    ),
)

또한 Unsloth의 'train_on_completions' 방법을 사용하여 assistant output만 학습하고 user input 손실을 무시합니다. 이는 미세 조정의 정확도를 높이는 데 도움이 됩니다!

In [ ]:
# user input은 학습하지 않고 assistant output만 학습하기 위해서 해당 부분 지정
from unsloth.chat_templates import train_on_responses_only
ft_trainer1 = train_on_responses_only(
    ft_trainer1,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=12):   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
# 대화가 토큰 형식으로 임베딩되어 ["input_ids"]에 저장되는데, 그걸 다시 디코딩 하는 코드
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

"<bos><bos><start_of_turn>user\n\nYou are a commercial district analysis expert specializing in Seoul, South Korea. \nYour primary task is to provide information on the monthly rent, deposit, area, and floor of properties that match the user's desired criteria. \nPlease provide responses to user inquiries in Korean.\n\n\n관악구에서 카페 매물 추천\n9000 / 80\n35평 이상<end_of_turn>\n<start_of_turn>model\n관악구 카페 창업 매물 정보 알려드릴게요.\n매물1\n- 보증금 : 2500 만원 \n- 월세 : 100 만원\n- 권리금 : 0 만원\n- 면적 : 158.7 m²\n- 층수 : 1층\n[매물 보러가기](link url)\n매물2\n- 보증금 : 1500 만원 \n- 월세 : 80 만원\n- 권리금 : 0 만원\n- 면적 : 149.39 m²\n- 층수 : 1층\n[매물 보러가기](link url)<end_of_turn>\n"

In [ ]:
# user_input을 마스킹처리하는 코드
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                         관악구 카페 창업 매물 정보 알려드릴게요.\n매물1\n- 보증금 : 2500 만원 \n- 월세 : 100 만원\n- 권리금 : 0 만원\n- 면적 : 158.7 m²\n- 층수 : 1층\n[매물 보러가기](link url)\n매물2\n- 보증금 : 1500 만원 \n- 월세 : 80 만원\n- 권리금 : 0 만원\n- 면적 : 149.39 m²\n- 층수 : 1층\n[매물 보러가기](link url)<end_of_turn>\n'

In [ ]:
# @title 현재 gpu 확인하는 코드
gpu_stats = torch.cuda.get_device_properties(0) # 현재 gpu 확인
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
4.52 GB of memory reserved.


In [ ]:
# @title 모델 학습
trainer_stats = ft_trainer1.train(resume_from_checkpoint = False) # 체크포인트부터 시작x

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 3 | Total steps = 39
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.964800
2,2.785800
3,3.331300
4,2.472600
5,2.263600
6,2.068000
7,1.565800
8,1.329800
9,1.393000
10,0.928700


In [ ]:
# 파인튜닝된 모델 객체
ft_model1 = ft_trainer1.model
ft_model1

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (vision_tower): SiglipVisionModel(
        (vision_model): SiglipVisionTransformer(
          (embeddings): SiglipVisionEmbeddings(
            (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
            (position_embedding): Embedding(4096, 1152)
          )
          (encoder): SiglipEncoder(
            (layers): ModuleList(
              (0-24): 25 x SiglipEncoderLayer(
                (self_attn): SiglipSdpaAttention(
                  (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                  (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
                )
                (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"{trainer_stats.metrics['train_runtime']}초 걸렸음.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)}분 걸렸음.")
print()
print(f"LoRA에 사용된 GPU메모리 = {used_memory_for_lora} GB.")
print(f"LoRA훈련에 사용된 GPU = {lora_percentage} %.")
print()
print(f"전체 훈련에 사용된 GPU 메모리 = {used_memory} GB.")
print(f"전체 훈련에 사용 GPU = {used_percentage} %.")

162.7412초 걸렸음.
2.71분 걸렸음.

LoRA에 사용된 GPU메모리 = 0.306 GB.
LoRA훈련에 사용된 GPU = 0.774 %.

전체 훈련에 사용된 GPU 메모리 = 4.826 GB.
전체 훈련에 사용 GPU = 12.2 %.


In [ ]:
def stream_answer(model,question):
  messages = [{
      "role": "user",
      "content": [{"type" : "text", "text" : question
    }]
  }]
  text = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt = True, # Must add for generation
  )

  _ = model.generate(
      **tokenizer([text], return_tensors = "pt").to("cuda"),
      max_new_tokens = 500, # Increase for longer outputs!
      # Recommended Gemma-3 settings!
      temperature = 1.0, top_p = 0.95, top_k = 64,
      streamer = TextStreamer(tokenizer, skip_prompt = True),
  )

In [ ]:
stream_answer(ft_model1, question)

갈현동 카페 창업 정보를 알려드릴게요.

**매물 1**
- 보증금: 1000만 원
- 월세: 100만 원
- 권리금: 1000만 원
- 면적: 50 m


In [ ]:
# 훈련 에포크 증가
ft_trainer2 = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",     # 데이터셋에서 대화가 담긴 열 이름
        per_device_train_batch_size = 2, # 장치(CPU, GPU, TPU)가 한 번에 학습하는 데이터 개수
        gradient_accumulation_steps = 4, # 그라디언트 4번을 가중 평균하여 한 번만 가중치 업데이트(2배치x4스탭 = 8배치와 같은 효과), gpu메모리 절약
        warmup_steps = 5,                # 훈련 초기에 학습률을 점진적으로 늘려 안정적인 학습 유도(0~5회 학습에서 점차 learning_rate 증가)
        num_train_epochs = 10,            # 전체 데이터를 모두 학습하는 횟수 (max_steps와 같이 넣으면, 빨리 끝나는 조건을 따름)
        # max_steps = 30,                # batch_size x maxsteps 만큼의 데이터양 학습 진행 (전체 데이터를 다 안 봐도 학습 횟수만큼만 학습하고 종료)
        learning_rate = 2e-4,            # 2e-4도 큰 폭임 긴 훈련은 2e-5처럼 더 작은 값을 사용
        logging_steps = 1,               # 훈련 상황 기록 빈도, 1이면 매 단계에 로그 기록
        optim = "adamw_8bit",            # 옵티마이저 - 아담w 8비트(메모리사용량을 줄이고 훈련속도를 높임)
        weight_decay = 0.01,             # 가중치를 감소시켜 과적합 방지
        lr_scheduler_type = "linear",    # 학습률 감소 방식(후반에 학습률을 줄여 최적값에 안정적으로 수렴)
        seed = 3407,                     # 랜덤스테이트같은거
        report_to = "none",              # 훈련 결과 확인 여부 "WandB"로 하면 Weights와 Baises 확인 가능
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",      # 체크포인트 저장 디렉토리
        save_strategy="best",            # 단계별로 저장
        metric_for_best_model="loss",
        save_steps=100,                  # 100단계마다 저장
        save_total_limit=3,              # 최대 3개의 체크포인트 저장
    ),
)

In [ ]:
ft_trainer2.train(resume_from_checkpoint = False)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 10 | Total steps = 130
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


Step,Training Loss
1,2.062300
2,1.901400
3,2.029500
4,1.558000
5,1.685300
6,1.529800
7,1.310000
8,1.305100
9,1.326000
10,0.961500


TrainOutput(global_step=130, training_loss=0.4607032890503223, metrics={'train_runtime': 385.2312, 'train_samples_per_second': 2.804, 'train_steps_per_second': 0.337, 'total_flos': 7194322074096000.0, 'train_loss': 0.4607032890503223})

In [ ]:
ft_model2 = ft_trainer2.model
stream_answer(ft_model2, question)

갈현동 카페 창업 정보를 알려드릴게요

매물1
- 보증금 : 1000만 원 
- 월세 : 100만 원
- 권리금 : 0원
- 면적 : 66.1 m²
- 층


In [ ]:
# LoRA수정을 위한 새로운 모델 불러오기
new_model, tokenizer = FastModel.from_pretrained(
    model_name = fourbit_models[1],
    max_seq_length = 4096,   # 모델이 처리할 수 있는 시퀀스 길이
    load_in_4bit = True,     # 4비트 양자화로 메모리 줄이기
    load_in_8bit = False,    # 8비트 양자화로 메모리는 4비트의 2배
    full_finetuning = False, # 풀파인튜닝 여부 LoRA를 사용할 거면 False
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:

# LoRA의 Rank 16으로 조정
lora16_model = FastModel.get_peft_model(
                new_model,
                finetune_vision_layers     = False, # 시각 기반인가? False
                finetune_language_layers   = True,  # 언어 기반인가? True
                finetune_attention_modules = True,  # Attention good for GRPO
                finetune_mlp_modules       = True,  # MLP (Multi-Layer Perceptron) 파인튜닝에 중요한 역할 항상 True 유지

                r = 16,            # LoRA 어댑터의 랭크(차원), 높을수록 모델의 설명력이 좋아지지만 과적합 됨(일반적으로 8이나 16)
                lora_alpha = 8,   # LoRA 어댑터의 스케일링(업데이트 크기), 보통 r과 같은값(최소가 r값)
                lora_dropout = 0, # LoRA 어댑터의 드롭아웃 비율(과적합 방지) 0이나 0.1
                bias = "none",    # LoRA어댑터의 편향 추가여부 "none", "lora_only"(lora에만 추가), "all" 모두 추가
                random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [ ]:
# rank증가
ft_trainer3 = SFTTrainer(
    model = lora16_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",     # 데이터셋에서 대화가 담긴 열 이름
        per_device_train_batch_size = 2, # 장치(CPU, GPU, TPU)가 한 번에 학습하는 데이터 개수
        gradient_accumulation_steps = 4, # 그라디언트 4번을 가중 평균하여 한 번만 가중치 업데이트(2배치x4스탭 = 8배치와 같은 효과), gpu메모리 절약
        warmup_steps = 5,                # 훈련 초기에 학습률을 점진적으로 늘려 안정적인 학습 유도(0~5회 학습에서 점차 learning_rate 증가)
        num_train_epochs = 20,            # 전체 데이터를 모두 학습하는 횟수 (max_steps와 같이 넣으면, 빨리 끝나는 조건을 따름)
        # max_steps = 30,                # batch_size x maxsteps 만큼의 데이터양 학습 진행 (전체 데이터를 다 안 봐도 학습 횟수만큼만 학습하고 종료)
        learning_rate = 2e-4,            # 2e-4도 큰 폭임 긴 훈련은 2e-5처럼 더 작은 값을 사용
        logging_steps = 1,               # 훈련 상황 기록 빈도, 1이면 매 단계에 로그 기록
        optim = "adamw_8bit",            # 옵티마이저 - 아담w 8비트(메모리사용량을 줄이고 훈련속도를 높임)
        weight_decay = 0.01,             # 가중치를 감소시켜 과적합 방지
        lr_scheduler_type = "linear",    # 학습률 감소 방식(후반에 학습률을 줄여 최적값에 안정적으로 수렴)
        seed = 3407,                     # 랜덤스테이트같은거
        report_to = "none",              # 훈련 결과 확인 여부 "WandB"로 하면 Weights와 Baises 확인 가능
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",      # 체크포인트 저장 디렉토리
        save_strategy="best",            # 단계별로 저장
        metric_for_best_model="loss",
        save_steps=100,                  # 100단계마다 저장
        save_total_limit=3,              # 최대 3개의 체크포인트 저장
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
ft_trainer3_state = ft_trainer3.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 20 | Total steps = 260
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,802,496/4,000,000,000 (0.75% trained)


Step,Training Loss
1,1.622200
2,1.606900
3,1.381600
4,1.070600
5,0.975500
6,0.815000
7,0.626100
8,0.529600
9,0.449500
10,0.302200


In [ ]:
ft_model3 = ft_trainer3.model
stream_answer(ft_model3, question)

갈현동 1층 카페 창업 매물 정보 알려드릴게요.

1.  매물 1
    *   보증금: 3000만 원
    *   월세: 200만 원
    *   권리금: 0원
    *   면적: 66.1m²
    *   지하 1층
    [매물 보러가기](link url)

2.  매물 2
    *   보증금: 2000만 원
    *   월세: 130만 원
    *   권리금: 0원
    *   면적: 59.5m²
    *   지하 1층
    [매물 보러가기](link url)

3.  매물 3
    *   보증금: 2000만 원
    *   월세: 170만 원
    *   권리금: 0원
    *   면적: 59.4m²
    *   지하 1층
    [매물 보러가기](link url)<end_of_turn>


In [ ]:
stream_answer(ft_model3, '창천동에서 축구교실 할 수 있을까요?')

네, 창천동에서 축구교실을 운영할 수 있습니다. 다만, 시설 임대료와 운영비용 등을 고려하여 안정적인 운영이 가능한지 검토해 보시는 것이 좋겠습니다.

축구교실 창업 정보를 확인해 보실 수 있는 곳은 다음과 같습니다:

*   **창업 정보 제공 사이트:**
    *   네이버 블로그: [https://blog.naver.com/PostList.board?user.log_screen_name=mylovefootball](https://blog.naver.com/PostList.board?user.log_screen_name=mylovefootball)
    *   클래스포트: [https://www.classfort.com/home/search/goods/listResult.do?searchKeyword=%EC%83%81%EA%B0%80%EB%A7%A4%EB%AC%BC%EC%83%81%EC%84%B8](https://www.classfort.com/home/search/goods/listResult.do?searchKeyword=%EC%83%81%EA%B0%80%EB%A7%A4%EB%AC%BC%EC%83%81%EC%84%B8)

추가적으로 궁금한 사항이 있다면 말씀해 주세요!<end_of_turn>


In [ ]:
# 학습횟수 증가
ft_trainer4 = SFTTrainer(
    model = lora16_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",     # 데이터셋에서 대}화가 담긴 열 이름
        per_device_train_batch_size = 2, # 장치(CPU, GPU, TPU)가 한 번에 학습하는 데이터 개수
        gradient_accumulation_steps = 4, # 그라디언트 4번을 가중 평균하여 한 번만 가중치 업데이트(2배치x4스탭 = 8배치와 같은 효과), gpu메모리 절약
        warmup_steps = 5,                # 훈련 초기에 학습률을 점진적으로 늘려 안정적인 학습 유도(0~5회 학습에서 점차 learning_rate 증가)
        num_train_epochs = 100,          # 전체 데이터를 모두 학습하는 횟수 (max_steps와 같이 넣으면, 빨리 끝나는 조건을 따름)
        # max_steps = 30,                # batch_size x maxsteps 만큼의 데이터양 학습 진행 (전체 데이터를 다 안 봐도 학습 횟수만큼만 학습하고 종료)
        learning_rate = 2e-4,            # 2e-4도 큰 폭임 긴 훈련은 2e-5처럼 더 작은 값을 사용
        logging_steps = 50,               # 훈련 상황 기록 빈도, 1이면 매 단계에 로그 기록
        optim = "adamw_8bit",            # 옵티마이저 - 아담w 8비트(메모리사용량을 줄이고 훈련속도를 높임)
        weight_decay = 0.01,             # 가중치를 감소시켜 과적합 방지
        lr_scheduler_type = "linear",    # 학습률 감소 방식(후반에 학습률을 줄여 최적값에 안정적으로 수렴)
        seed = 3407,                     # 랜덤스테이트같은거
        report_to = "none",              # 훈련 결과 확인 여부 "WandB"로 하면 Weights와 Baises 확인 가능
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",      # 체크포인트 저장 디렉토리
        save_strategy="best",            # 단계별로 저장
        metric_for_best_model="loss",
        save_steps=100,                  # 100단계마다 저장
        save_total_limit=3,              # 최대 3개의 체크포인트 저장
    ),
)

In [ ]:
ft_trainer4.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 100 | Total steps = 1,300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,802,496/4,000,000,000 (0.75% trained)


Step,Training Loss
50,0.021900
100,0.016000
150,0.012400
200,0.012000
250,0.012300
300,0.013000
350,0.011900
400,0.011000
450,0.010800
500,0.010800


TrainOutput(global_step=1300, training_loss=0.011660424012404223, metrics={'train_runtime': 3705.4809, 'train_samples_per_second': 2.915, 'train_steps_per_second': 0.351, 'total_flos': 1.7241779500044403e+17, 'train_loss': 0.011660424012404223})

In [ ]:
ft_model4 = ft_trainer4.model
stream_answer(ft_model4, question)

갈현동 1층 카페 창업 매물 정보 알려드릴게요.

1.  매물 1
    *   보증금: 3000만 원
    *   월세: 200만 원
    *   권리금: 0원
    *   면적: 66.1m²
    *   지하 1층
    [매물 보러가기](link url)

2.  매물 2
    *   보증금: 2000만 원
    *   월세: 130만 원
    *   권리금: 0원
    *   면적: 59.5m²
    *   지하 1층
    [매물 보러가기](link url)

3.  매물 3
    *   보증금: 2000만 원
    *   월세: 170만 원
    *   권리금: 0원
    *   면적: 59.4m²
    *   지하 1층
    [매물 보러가기](link url)<end_of_turn>


In [ ]:
stream_answer(ft_model4, '창천동에서 축구교실 할 수 있을까요?')

네, 창천동에서 축구교실을 운영할 수 있습니다. 다만, 시설 임대료와 운영비용 등을 고려하여 안정적인 운영이 가능한<bos><start_of_turn>user
창천동에서 축구교실 할 수 있을까요?<end_of_turn>
<start_of_turn>model
네, 창천동에서 축구교실을 운영할 수 있습니다. 다만, 시설 임대료와 운영비용 등을 고려하여 안정적인 운영이 가능한지 검토해 보시는 것이 좋겠습니다.

축구교실 창업 정보를 확인해 보실 수 있는 곳은 다음과 같습니다:

*   **창업 정보 제공 사이트:**
    *   네이버 블로그: 

KeyboardInterrupt: 

In [ ]:
# 학습횟수 증가
ft_trainer5 = SFTTrainer(
    model = lora16_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",     # 데이터셋에서 대화가 담긴 열 이름
        per_device_train_batch_size = 2, # 장치(CPU, GPU, TPU)가 한 번에 학습하는 데이터 개수
        gradient_accumulation_steps = 4, # 그라디언트 4번을 가중 평균하여 한 번만 가중치 업데이트(2배치x4스탭 = 8배치와 같은 효과), gpu메모리 절약
        warmup_steps = 5,                # 훈련 초기에 학습률을 점진적으로 늘려 안정적인 학습 유도(0~5회 학습에서 점차 learning_rate 증가)
        num_train_epochs = 200,          # 전체 데이터를 모두 학습하는 횟수 (max_steps와 같이 넣으면, 빨리 끝나는 조건을 따름)
        # max_steps = 30,                # batch_size x maxsteps 만큼의 데이터양 학습 진행 (전체 데이터를 다 안 봐도 학습 횟수만큼만 학습하고 종료)
        learning_rate = 2e-4,            # 2e-4도 큰 폭임 긴 훈련은 2e-5처럼 더 작은 값을 사용
        logging_steps = 100,               # 훈련 상황 기록 빈도, 1이면 매 단계에 로그 기록
        optim = "adamw_8bit",            # 옵티마이저 - 아담w 8비트(메모리사용량을 줄이고 훈련속도를 높임)
        weight_decay = 0.01,             # 가중치를 감소시켜 과적합 방지
        lr_scheduler_type = "linear",    # 학습률 감소 방식(후반에 학습률을 줄여 최적값에 안정적으로 수렴)
        seed = 3407,                     # 랜덤스테이트같은거
        report_to = "none",              # 훈련 결과 확인 여부 "WandB"로 하면 Weights와 Baises 확인 가능
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",      # 체크포인트 저장 디렉토리
        save_strategy="best",            # 단계별로 저장
        metric_for_best_model="loss",
        save_steps=100,                  # 100단계마다 저장
        save_total_limit=3,              # 최대 3개의 체크포인트 저장
    ),
)

In [ ]:
ft_model5 = ft_trainer5.model
stream_answer(ft_model5, question)

갈현동 카페 창업 정보를 알려드릴게요.

현재 갈현동에서 보증금 5000만 원, 월세 200만 원 이하, 권리금 0원, 1층, 50 m² 이상의 카페 창업 매물을 추천해 드릴게요.

매물1
- 보증금 : 2000만 원
- 월세 : 120만 원
- 권리금 : 0원
- 면적 : 59.5m²
- 특징 : 깔끔한 내부 인테리어, 무권리금으로 초기 비용 부담 적음.
- 매물 보러가기 : [https://www.zigbang.com/home/store/items/571420](https://www.zigbang.com/home/store/items/571420)

매물2
- 보증금 : 3000만 원
- 월세 : 180만 원
- 권리금 : 0원
- 면적 : 66.9m²
- 특징 : 넓은 공간과 좋은 위치, 다양한 업종 활용 가능.
- 매물 보러가기 : [https://www.zigbang.com/home/store/items/515778](https://www.zigbang.com/home/store/items/515778)

매물3
- 보증금 : 3000만 원
- 월세 : 160만 원
- 권리금 : 0원
- 면적 : 56.8 m²
- 특징 : 대로변 상가로 접근성 좋음.
- 매물 보러가기 : [https://www.zigbang.com/home/store/items/515778](https://www.zigbang.com/home/store/items/515778)

더 필요한 정보가 있으시면 말씀해주세요!<end_of_turn>


In [ ]:
stream_answer(ft_model5, '창천동에서 축구교실 할 수 있을까요?')

네, 창천동에서 축구교실을 운영하는 곳이 있어요!

현재 축구장과 함께 축구교실을 운영하는 곳이 2군데 정도 있어요.

*   **창천동 축구장:** 010-2200-2777
*   **창천동 축구클럽:** 010-7788-2222

더 자세한 정보가 필요하시면 말씀해주세요!<end_of_turn>


In [ ]:
# 4b모델 -> 12b모델
model_12b, tokenizer = FastModel.from_pretrained(
    model_name = fourbit_models[2],
    max_seq_length = 4096,   # 모델이 처리할 수 있는 시퀀스 길이
    load_in_4bit = True,     # 4비트 양자화로 메모리 줄이기
    load_in_8bit = False,    # 8비트 양자화로 메모리는 4비트의 2배
    full_finetuning = False, # 풀파인튜닝 여부 LoRA를 사용할 거면 False
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/259k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.01G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
# LoRA의 Rank 16으로 조정
lora8_12b_model = FastModel.get_peft_model(
                model_12b,
                finetune_vision_layers     = False, # 시각 기반인가? False
                finetune_language_layers   = True,  # 언어 기반인가? True
                finetune_attention_modules = True,  # Attention good for GRPO
                finetune_mlp_modules       = True,  # MLP (Multi-Layer Perceptron) 파인튜닝에 중요한 역할 항상 True 유지

                r = 8,            # LoRA 어댑터의 랭크(차원), 높을수록 모델의 설명력이 좋아지지만 과적합 됨(일반적으로 8이나 16)
                lora_alpha = 8,   # LoRA 어댑터의 스케일링(업데이트 크기), 보통 r과 같은값(최소가 r값)
                lora_dropout = 0, # LoRA 어댑터의 드롭아웃 비율(과적합 방지) 0이나 0.1
                bias = "none",    # LoRA어댑터의 편향 추가여부 "none", "lora_only"(lora에만 추가), "all" 모두 추가
                random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [ ]:
# 12b모델 사용
ft_trainer6 = SFTTrainer(
    model = lora8_12b_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",     # 데이터셋에서 대화가 담긴 열 이름
        per_device_train_batch_size = 2, # 장치(CPU, GPU, TPU)가 한 번에 학습하는 데이터 개수
        gradient_accumulation_steps = 4, # 그라디언트 4번을 가중 평균하여 한 번만 가중치 업데이트(2배치x4스탭 = 8배치와 같은 효과), gpu메모리 절약
        warmup_steps = 5,                # 훈련 초기에 학습률을 점진적으로 늘려 안정적인 학습 유도(0~5회 학습에서 점차 learning_rate 증가)
        num_train_epochs = 10,          # 전체 데이터를 모두 학습하는 횟수 (max_steps와 같이 넣으면, 빨리 끝나는 조건을 따름)
        # max_steps = 30,                # batch_size x maxsteps 만큼의 데이터양 학습 진행 (전체 데이터를 다 안 봐도 학습 횟수만큼만 학습하고 종료)
        learning_rate = 2e-4,            # 2e-4도 큰 폭임 긴 훈련은 2e-5처럼 더 작은 값을 사용
        logging_steps = 100,               # 훈련 상황 기록 빈도, 1이면 매 단계에 로그 기록
        optim = "adamw_8bit",            # 옵티마이저 - 아담w 8비트(메모리사용량을 줄이고 훈련속도를 높임)
        weight_decay = 0.01,             # 가중치를 감소시켜 과적합 방지
        lr_scheduler_type = "linear",    # 학습률 감소 방식(후반에 학습률을 줄여 최적값에 안정적으로 수렴)
        seed = 3407,                     # 랜덤스테이트같은거
        report_to = "none",              # 훈련 결과 확인 여부 "WandB"로 하면 Weights와 Baises 확인 가능
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",      # 체크포인트 저장 디렉토리
        save_strategy="epoch",            # 단계별로 저장
        metric_for_best_model="loss",
        save_steps=100,                  # 100단계마다 저장
        save_total_limit=3,              # 최대 3개의 체크포인트 저장
    ),
)

In [ ]:
ft_trainer6.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 10 | Total steps = 130
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 32,735,232/12,000,000,000 (0.27% trained)


Step,Training Loss
100,0.367300


TrainOutput(global_step=130, training_loss=0.2982967871886033, metrics={'train_runtime': 663.393, 'train_samples_per_second': 1.628, 'train_steps_per_second': 0.196, 'total_flos': 5.572547396904672e+16, 'train_loss': 0.2982967871886033})

In [ ]:
ft_model6 = ft_trainer6.model
stream_answer(ft_model6, question)

갈현동 카페 창업 정보를 알려드릴게요.

현재 갈현동에서 보증금 5000만 원, 월세 200만 원 이하, 권리금 0원, 1층, 50 m² 이상의 카페 창업 매물을 추천해 드릴게요.

매물1
- 보증금 : 2000만 원
- 월세 : 120만 원
- 권리금 : 0원
- 면적 : 59.5m²
- 특징 : 깔끔한 내부 인테리어, 무권리금으로 초기 비용 부담 적음.
- 매물 보러가기 : [https://www.zigbang.com/home/store/items/571420](https://www.zigbang.com/home/store/items/571420)

매물2
- 보증금 : 3000만 원
- 월세 : 180만 원
- 권리금 : 0원
- 면적 : 66.9m²
- 특징 : 넓은 공간과 좋은 위치, 다양한 업종 활용 가능.
- 매물 보러가기 : [https://www.zigbang.com/home/store/items/515778](https://www.zigbang.com/home/store/items/515778)

매물3
- 보증금 : 3000만 원
- 월세 : 160만 원
- 권리금 : 0원
- 면적 : 56.8 m²
- 특징 : 대로변 상가로 접근성 좋음.
- 매물 보러가기 : [https://www.zigbang.com/home/store/items/515778](https://www.zigbang.com/home/store/items/515778)

더 필요한 정보가 있으시면 말씀해주세요!<end_of_turn>


In [ ]:
stream_answer(ft_model6, '갈현동에 월세30만원 이하, 보증금 3000만원 이하인 매물 있나요?')

갈현동에서 월세 30만원 이하, 보증금 3000만원 이하인 매물을 찾는다면, 주로 작은 규모의 상가나 매장을 고려해야 할 것 같아요. 

현재 갈현동에 딱 맞는 매물은 아니지만, 비슷한 조건의 매물 추천해 드릴게요.

1.  **월세:** 30만원 ~ 50만원
    **보증금:** 1000만원 ~ 2500만원
    **면적:** 16.6m² ~ 36.2m²
    [매물 보러가기](link url="https://www.zigbang.com/home/store/items/571422?log_screen_name=%EC%83%81%EA%B0%80%EB%A7%A4%EB%AC%BC%EC%83%81%EC%84%B8")

2.  **월세:** 30만원
    **보증금:** 1000만원
    **면적:** 23.1 m²
    [매물 보러가기](link url="https://www.zigbang.com/home/store/items/517277?log_screen_name=%EC%83%81%EA%B0%80%EB%A7%A4%EB%AC%BC%EC%83%81%EC%84%B8")

3.  **월세:** 30만원
    **보증금:** 1500만원
    **면적:** 19.8 m²
    [매물 보러가기](link url="https://www.zigbang.com/home/store/items/517277?log_screen_name=%EC%83%81%EA%B0%80%EB%A7%A4%EB%AC%BC%EC%83%81%EC%84%B8")

이 외에도 다양한 매물이 있을 수 있으니, 부동산 앱이나 웹사이트에서 갈현동 상가 입지에 대한 정보를 확인해 보시는 것을 추천드려요!


In [ ]:
# 12b모델 사용
ft_trainer7 = SFTTrainer(
    model = lora8_12b_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",     # 데이터셋에서 대화가 담긴 열 이름
        per_device_train_batch_size = 2, # 장치(CPU, GPU, TPU)가 한 번에 학습하는 데이터 개수
        gradient_accumulation_steps = 4, # 그라디언트 4번을 가중 평균하여 한 번만 가중치 업데이트(2배치x4스탭 = 8배치와 같은 효과), gpu메모리 절약
        warmup_steps = 5,                # 훈련 초기에 학습률을 점진적으로 늘려 안정적인 학습 유도(0~5회 학습에서 점차 learning_rate 증가)
        num_train_epochs = 50,          # 전체 데이터를 모두 학습하는 횟수 (max_steps와 같이 넣으면, 빨리 끝나는 조건을 따름)
        # max_steps = 30,                # batch_size x maxsteps 만큼의 데이터양 학습 진행 (전체 데이터를 다 안 봐도 학습 횟수만큼만 학습하고 종료)
        learning_rate = 2e-4,            # 2e-4도 큰 폭임 긴 훈련은 2e-5처럼 더 작은 값을 사용
        logging_steps = 100,               # 훈련 상황 기록 빈도, 1이면 매 단계에 로그 기록
        optim = "adamw_8bit",            # 옵티마이저 - 아담w 8비트(메모리사용량을 줄이고 훈련속도를 높임)
        weight_decay = 0.01,             # 가중치를 감소시켜 과적합 방지
        lr_scheduler_type = "linear",    # 학습률 감소 방식(후반에 학습률을 줄여 최적값에 안정적으로 수렴)
        seed = 3407,                     # 랜덤스테이트같은거
        report_to = "none",              # 훈련 결과 확인 여부 "WandB"로 하면 Weights와 Baises 확인 가능
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",      # 체크포인트 저장 디렉토리
        save_strategy="epoch",            # 단계별로 저장
        metric_for_best_model="loss",
        save_steps=100,                  # 100단계마다 저장
        save_total_limit=3,              # 최대 3개의 체크포인트 저장
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
ft_trainer7.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 50 | Total steps = 650
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 32,735,232/12,000,000,000 (0.27% trained)


Step,Training Loss
100,0.048700
200,0.016300
300,0.013200
400,0.012700
500,0.012700
600,0.012400


TrainOutput(global_step=650, training_loss=0.01878329111979558, metrics={'train_runtime': 3294.3992, 'train_samples_per_second': 1.639, 'train_steps_per_second': 0.197, 'total_flos': 2.6448937333830432e+17, 'train_loss': 0.01878329111979558})

In [ ]:
ft_model7 = ft_trainer7.model
stream_answer(ft_model7, question)

갈현동 카페 창업 정보를 알려드릴게요.

현재 갈현동에서 보증금 5000만 원 이하, 월세 200만 원 이하, 권리금 0원, 1층, 50 m² 이상의 매물을 기준으로 카페 창업에 적합한 곳을 추천해 드릴게요.

매물1
- 보증금 : 0 만원 
- 월세 : 100 만원
- 권리금 : 0 만원
- 면적 : 56.2 m²
- 위치 : 갈현동 1층
- 설명 : 저렴한 월세로 카페 창업이 가능하며, 학원 근처에 위치하여 학생들을 타겟으로 영업이 가능합니다.

매물2
- 보증금 : 2000 만원 
- 월세 : 100 만원
- 권리금 : 0 만원
- 면적 : 56.2 m²
- 위치 : 갈현동 1층
- 설명 : 넓은 공간과 저렴한 월세로 다양한 아이템 운영이 가능합니다.

매물3
- 보증금 : 2000 만원 
- 월세 : 100 만원
- 권리금 : 0 만원
- 면적 : 56.2 m²
- 위치 : 갈현동 1층
- 설명 : 주거지역에 위치하여 주민들의 유입이 꾸준히 있으며, 카페뿐만 아니라 진료개원을 고려할 수 있는 매물입니다.

더 필요한 정보가 있으시면 말씀해주세요!<end_of_turn>


In [ ]:
stream_answer(ft_model7, '창천동에서 축구교실 창업할 매물 있을까요?')

창천동에서 축구교실을 창업하기 적합한 매물을 찾는다면, 중구 매물 추천 링크를 알려드릴게요.

📍 위치: 중구 창천동
💰 보증금: 2000만 원 ~ 3000만 원
🏠 면적: 100 m² 이상
🚗 접근성: 대중교통 접근성이 좋고, 주차 공간이 확보된 곳
시설: 실내 축구장 또는 운동장이 마련되어 있거나, 시설 설치가 가능한 곳

현재 창천동에서 축구교실 창업 가능성이 높은 매물은 없지만, 계속해서 새로운 매물이 등록되는 것을 확인하는 것이 좋습니다.<end_of_turn>


In [ ]:
# 12b모델 사용
ft_trainer8 = SFTTrainer(
    model = lora8_12b_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",     # 데이터셋에서 대화가 담긴 열 이름
        per_device_train_batch_size = 2, # 장치(CPU, GPU, TPU)가 한 번에 학습하는 데이터 개수
        gradient_accumulation_steps = 4, # 그라디언트 4번을 가중 평균하여 한 번만 가중치 업데이트(2배치x4스탭 = 8배치와 같은 효과), gpu메모리 절약
        warmup_steps = 5,                # 훈련 초기에 학습률을 점진적으로 늘려 안정적인 학습 유도(0~5회 학습에서 점차 learning_rate 증가)
        num_train_epochs = 130,          # 전체 데이터를 모두 학습하는 횟수 (max_steps와 같이 넣으면, 빨리 끝나는 조건을 따름)
        # max_steps = 30,                # batch_size x maxsteps 만큼의 데이터양 학습 진행 (전체 데이터를 다 안 봐도 학습 횟수만큼만 학습하고 종료)
        learning_rate = 2e-4,            # 2e-4도 큰 폭임 긴 훈련은 2e-5처럼 더 작은 값을 사용
        logging_steps = 100,             # 훈련 상황 기록 빈도, 1이면 매 단계에 로그 기록
        optim = "adamw_8bit",            # 옵티마이저 - 아담w 8비트(메모리사용량을 줄이고 훈련속도를 높임)
        weight_decay = 0.01,             # 가중치를 감소시켜 과적합 방지
        lr_scheduler_type = "linear",    # 학습률 감소 방식(후반에 학습률을 줄여 최적값에 안정적으로 수렴)
        seed = 3407,                     # 랜덤스테이트같은거ㅌ
        report_to = "none",              # 훈련 결과 확인 여부 "WandB"로 하면 Weights와 Baises 확인 가능
        output_dir="/content/drive/MyDrive/Colab Notebooks/checkpoint",      # 체크포인트 저장 디렉토리
        save_strategy="epoch",            # 단계별로 저장
        metric_for_best_model="loss",
        save_steps=100,                  # 100단계마다 저장
        save_total_limit=3,              # 최대 3개의 체크포인트 저장
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
ft_trainer8.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 108 | Num Epochs = 130 | Total steps = 1,690
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 32,735,232/12,000,000,000 (0.27% trained)


Step,Training Loss
100,0.019500
200,0.013800
300,0.013200
400,0.013100
500,0.013000
600,0.012900
700,0.012700
800,0.013100
900,0.012900
1000,0.012900


TrainOutput(global_step=1690, training_loss=0.013240145014587944, metrics={'train_runtime': 8663.6956, 'train_samples_per_second': 1.621, 'train_steps_per_second': 0.195, 'total_flos': 6.876389335928891e+17, 'train_loss': 0.013240145014587944})

In [ ]:
ft_model8 = ft_trainer8.model
stream_answer(ft_model8, question)

갈현동에서 카페 창업에 적합한 매물을 찾는다면, 중구는 어떤가요?<end_of_turn>


In [ ]:
stream_answer(ft_model8, '창천동에서 축구교실 창업할 매물 있을까요?')

창천동에서 축구교실 창업하기 좋은 매물을 찾는다면, 중구는 물론이고 인근 지역의 매물 정보도 함께 확인하는 것이 좋습니다. 

현재 창천동에 축구교실에 적합한 매물은 없지만, 인근 지역에 적합한 매물이 있을 수 있습니다. 예를 들어, 남구의 경우 지하 100m² 이상의 매물이 2개나 있어 확인해볼만한 가치가 있습니다.

더 필요한 정보가 있다면 말씀해주세요!<end_of_turn>


LoRA의 Adapter만 저장하는 방법

In [ ]:
## 로컬 저장
# model.save_pretrained("gemma-3")  # Local saving
# tokenizer.save_pretrained("gemma-3")

# 환경 변수에서 액세스 토큰 가져오기
from google.colab import userdata
hg_access_token = userdata.get('hf_token')

# 허깅페이스에 저장
ft_model5.push_to_hub("Envy1025/Retail_Property_Finder_Chatbot", token = hg_access_token) # Online saving
tokenizer.push_to_hub("Envy1025/Retail_Property_Finder_Chatbot", token = hg_access_token) # Online saving

README.md:   0%|          | 0.00/599 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/119M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Envy1025/Retail_Property_Finder_Chatbot


  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 500, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down what Gemma-3 is. It's a fascinating development in the world of AI, and here's a comprehensive overview:

**1. What it is:**

* **A Family of Open-Weight Language Models:** Gemma-3 isn't just *one* model


### VLLM을 위해 float16으로 저장하기

우리는 배포를 위해 'float16'에 직접 저장하는 것도 지원합니다! 우리는 이를 'gemma-3-finetune' 폴더에 저장합니다. 실행할 수 있도록 '만약 거짓이라면'을 '만약 참이라면'으로 설정하세요!

In [ ]:
# # @title 로컬에 파인튜닝 모델 저장
# if False: # Change to True to save finetune!
#     ft_model7.save_pretrained_merged("Retail_Property_Finder_Chatbot", tokenizer)

In [ ]:
# @title 파인튜닝 모델 허깅페이스에 푸쉬
if True: # Change to True to upload finetune
    ft_model8.push_to_hub_merged(
        "Envy1025/Retail_Property_Finder_Chatbot_12b", tokenizer,
        token = hg_access_token
    )

  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/109k [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/5 [00:00<?, ?it/s]

...

Unsloth: Merging weights into 16bit:  80%|████████  | 4/5 [05:51<01:28, 88.15s/it]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 5/5 [07:17<00:00, 87.59s/it]


In [ ]:
# @title 허깅페이스에서 모델 불러오기
hug_model, tokenizer = FastModel.from_pretrained(
    "Envy1025/Retail_Property_Finder_Chatbot",  # Hugging Face Hub의 모델 ID
    max_seq_length=4096,
    load_in_4bit=True,
    load_in_8bit=False,
    full_finetuning=False,
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/119M [00:00<?, ?B/s]

In [ ]:
# @title 허깅페이스에서 불러온 모델 사용
question = "갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘"
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : question,
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # 대화형 모델에게 생성을 알리는 신호
)
outputs = hug_model.generate(
    # 토크나이저가 텍스트를 숫자로 변환, return_tensors='pt'는 PyTorch텐서로 변환
    **tokenizer([text], return_tensors = "pt").to("cuda"), # to.("cuda")는 PyTorch텐서를 CUDA(GPU)장치로 이동
    max_new_tokens = 500, # 출력 길이
    # Gemma-3의 권장 설정
    temperature = 1.0, top_p = 0.95, top_k = 64,
)

# 출력 결과 확인
response = tokenizer.batch_decode(outputs)

user_input = re.search(r'<start_of_turn>user\n(.*?)<end_of_turn>', response[0]).group(1).strip()
print(f"""질문 : \n {user_input}""")

print(response[0][response[0].find('\n<start_of_turn>model\n')+len('\n<start_of_turn>model\n'):])

질문 : 
 갈현동에서 카페 창업을 하려고해. 보증금 5000만에 월세 200만 원 이하로 권리금은 없었으면 좋겠고 1층에  50 m² 이상으로 찾아줘
갈현동 1층 카페 창업 매물 정보 알려드릴게요.

1.  매물 1
    *   보증금: 3000만 원
    *   월세: 200만 원
    *   권리금: 0원
    *   면적: 66.1m²
    *   지하 1층
    [매물 보러가기](link url)

2.  매물 2
    *   보증금: 2000만 원
    *   월세: 130만 원
    *   권리금: 0원
    *   면적: 59.5m²
    *   지하 1층
    [매물 보러가기](link url)

3.  매물 3
    *   보증금: 2000만 원
    *   월세: 170만 원
    *   권리금: 0원
    *   면적: 59.4m²
    *   지하 1층
    [매물 보러가기](link url)<end_of_turn>


### GGUF / lama.cpp 변환
'GGUF' / 'llama.cpp'로 저장하기 위해 모든 모델에서 기본적으로 지원합니다! 현재로서는 'Q8_0, F16 또는 BF16' 정밀도로 쉽게 변환할 수 있습니다. 4비트용 'Q4_K_M'은 나중에 출시될 예정입니다!

In [ ]:
# if False: # Change to True to save to GGUF
#     model.save_pretrained_gguf(
#         "gemma-3-finetune",
#         quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
#     )

In [ ]:
# if False: # Change to True to upload GGUF
#     model.push_to_hub_gguf(
#         "gemma-3-finetune",
#         quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
#         repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
#         token = "hf_...",
#     )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
